# How to train a COITE ensemble on FMNIST
> See how

In [ ]:
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split

data = fetch_openml("Fashion-MNIST", data_home="/home/matthieu/sklearn_data")

X = data["data"].reshape(-1,28,28)
Y = data["target"]

xtrain, xtest, ytrain, ytest = train_test_split(X, Y, train_size=60000, random_state=13)

In [ ]:
from mlg_lib.imgfeat import HogFactory, DaisyFactory, flatten, LBPFactory
from mlg_lib.ml_utils import make_single_feature_model, sk_train

from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.ensemble import ExtraTreesClassifier, VotingClassifier

In [ ]:
FEATURES = [
    HogFactory(orientations=8, pixels_per_cell=(8, 8),cells_per_block=(1, 1), multichannel=False),
    DaisyFactory(orientations=8, step=8, radius=8, rings=2),
    flatten,
    LBPFactory(radius=8)
]

FEATURE_NAMES = [
    "HOG",
    "DAISY",
    "RAW",
    "LBP"
]

In [ ]:
pipeline = VotingClassifier(
    list(
        zip(
            FEATURE_NAMES,
        list( 
            map(lambda f: make_single_feature_model(f, ExtraTreesClassifier(n_estimators=25)), FEATURES) )
        )
    ),
    voting="soft"
)

In [ ]:
out = sk_train(xtrain, xtest, ytrain, ytest, pipeline, metrics=dict(cm=confusion_matrix, acc=accuracy_score).items())

In [ ]:
out

TrainingOutput(model=VotingClassifier(estimators=[('HOG',
                              Pipeline(memory=None,
                                       steps=[('lambdarow',
                                               LambdaRow(row_func=HogFactory(orientations=8, pixels_per_cell=(8, 8), cells_per_block=(1, 1), multichannel=False))),
                                              ('extratreesclassifier',
                                               ExtraTreesClassifier(bootstrap=False,
                                                                    class_weight=None,
                                                                    criterion='gini',
                                                                    max_depth=None,
                                                                    max_features='auto',
                                                                    max_leaf_nodes=None,
                                                                    min_imp

In [ ]:
from sklearn.ensemble import RandomForestClassifier

pipeline = VotingClassifier(
    list(
        zip(
            FEATURE_NAMES,
        list( 
            map(lambda f: make_single_feature_model(f, RandomForestClassifier(n_estimators=50, max_features="log2", max_depth=None, n_jobs=-1)), FEATURES) )
        )
    ),
    voting="soft"
)

In [ ]:
out = sk_train(xtrain, xtest, ytrain, ytest, pipeline, metrics=dict(cm=confusion_matrix, acc=accuracy_score).items())

In [ ]:
out

TrainingOutput(model=VotingClassifier(estimators=[('HOG',
                              Pipeline(memory=None,
                                       steps=[('lambdarow',
                                               LambdaRow(row_func=HogFactory(orientations=8, pixels_per_cell=(8, 8), cells_per_block=(1, 1), multichannel=False))),
                                              ('randomforestclassifier',
                                               RandomForestClassifier(bootstrap=True,
                                                                      class_weight=None,
                                                                      criterion='gini',
                                                                      max_depth=None,
                                                                      max_features='log2',
                                                                      max_leaf_nodes=None,
                                                              